Dataset: https://www.kaggle.com/datasets/thedevastator/dailydialog-multi-turn-dialog-with-intention-and

# Classic Seq2Seq

In [1]:
import pandas as pd
import numpy as np
import ast
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, LSTM, Embedding, Dense

In [52]:
pairs=[
    ("Hi!", "Hey there!"),
    ("How's it going?", "Pretty good, thanks!"),
    ("What's up?", "Not much, just chilling."),
    ("Good morning!", "Morning! Hope your day's off to a great start."),
    ("Hey, how are you?", "I'm doing great, you?"),
    ("What's the weather like?", "It's sunny and warm out there!"),
    ("How old are you?", "I'm forever young, how about you?"),
    ("What do you do for fun?", "I love chatting and learning new things!"),
    ("Bye for now!", "Catch you later!"),
    ("How's your day been?", "It's been awesome, thanks for asking!"),
    ("Hello there!", "Hi! Nice to hear from you."),
    ("What's your favorite hobby?", "I enjoy exploring new topics, you?"),
    ("Is it raining outside?", "Nope, clear skies today!"),
    ("How are you feeling?", "Feeling great, thanks for asking!"),
    ("Good night!", "Sweet dreams!"),
    ("What's new with you?", "Just hanging out, ready to chat!"),
    ("How's the weather today?", "It's a bit cloudy, but nice!"),
    ("Do you like to read?", "I love diving into new info, you?"),
    ("See you soon!", "Later!"),
    ("How's everything going?", "All good here, what's up with you?"),
    ("Hi, how's it going?", "Hey! I'm doing well, thanks."),
    ("What's the forecast?", "Looks like a sunny day ahead!"),
    ("What do you enjoy doing?", "I like helping out and learning!"),
    ("Take care!", "You too, bye!"),
    ("How are you today?", "I'm super, thanks for asking!"),
    ("Hey there!", "Yo! What's good?"),
    ("Is it cold outside?", "A bit chilly, grab a jacket!"),
    ("What's your favorite thing to do?", "Chatting with folks like you!"),
    ("Goodbye!", "See ya!"),
    ("How's your mood?", "I'm in high spirits, you?"),
    ("Hi, what's up?", "Not much, just here for you!"),
    ("How's the weather out there?", "Bright and sunny, perfect day!"),
    ("Do you have any hobbies?", "I love exploring knowledge, you?"),
    ("Catch you later!", "Until next time!"),
    ("How you holding up?", "Doing great, thanks for asking!"),
    ("Hello!", "Hey, good to see you!"),
    ("Is it nice outside?", "Yeah, it's gorgeous out there!"),
    ("What do you like to do?", "I enjoy a good chat, you?"),
    ("Have a good one!", "You too!"),
    ("How's your day going?", "Pretty smooth, how about yours?"),
    ("Hey, what's good?", "All good here, what's up?"),
    ("What's the weather like today?", "It's warm and breezy!"),
    ("Got any fun hobbies?", "I like learning new stuff, you?"),
    ("See you around!", "Later, friend!"),
    ("How you feeling today?", "I'm on top of the world!"),
    ("Hi there!", "Hey! What's up?"),
    ("Is it sunny out?", "Yup, sun's shining bright!"),
    ("What do you do in your free time?", "I love chatting, what's your thing?"),
    ("Bye bye!", "Take care!"),
    ("How's it hanging?", "All good, how about you?"),
    ("Good afternoon!", "Afternoon! Hope you're having a great day."),
    ("What's the weather like now?", "It's nice and cool outside!"),
    ("What are your hobbies?", "I'm into exploring ideas, you?"),
    ("Talk later!", "Sounds good, bye!"),
    ("How are you doing?", "I'm awesome, thanks!"),
    ("Hey, how's it going?", "Yo! I'm good, you?"),
    ("Is it warm today?", "Yeah, it's toasty out there!"),
    ("What do you enjoy?", "I like helping out, what's your hobby?"),
    ("See ya later!", "Until next time!"),
    ("How's your vibe today?", "Super chill, you?"),
    ("Hello, friend!", "Hey! Great to hear from you."),
    ("What's the weather doing?", "It's partly cloudy, nice day!"),
    ("Got any cool hobbies?", "I love learning, what's yours?"),
    ("Have a great day!", "You too, bye!"),
    ("How you doing today?", "I'm fantastic, thanks!"),
    ("Hi, what's new?", "Not much, just here for you!"),
    ("Is it nice out today?", "Yeah, perfect weather!"),
    ("What do you like doing?", "I enjoy a good convo, you?"),
    ("Catch ya later!", "Bye for now!"),
    ("How's your day been so far?", "It's been great, you?"),
    ("Hey there, what's up?", "Yo! Just chilling, you?"),
    ("What's the weather like out there?", "Sunny and warm, love it!"),
    ("Any fun hobbies?", "I like exploring new topics, you?"),
    ("Take it easy!", "You too, later!"),
    ("How you feeling?", "I'm great, thanks for asking!"),
    ("Good morning, friend!", "Morning! Hope you're ready for a great day!"),
    ("Is it rainy today?", "Nope, it's clear and bright!"),
    ("What do you do for fun?", "I love chatting, what's your thing?"),
    ("See you soon!", "Later!"),
    ("How's everything?", "All good here, you?"),
    ("Hi, how you doing?", "Hey! I'm awesome, thanks!"),
    ("What's the weather today?", "It's cool and sunny!"),
    ("What are your interests?", "I like learning new stuff, you?"),
    ("Bye for now!", "Catch you later!"),
    ("How's your day?", "Going great, thanks!"),
    ("Hey, what's up?", "Not much, just here!"),
    ("Is it hot outside?", "A bit warm, but nice!"),
    ("What's your favorite activity?", "I enjoy helping out, you?"),
    ("Have a good night!", "Sweet dreams!"),
    ("How you holding up today?", "I'm good, you?"),
    ("Hello there!", "Hey! Nice to see you!"),
    ("What's the weather like?", "It's sunny and pleasant!"),
    ("Got any hobbies?", "I love a good chat, you?"),
    ("Talk soon!", "Later!"),
    ("How's it going today?", "I'm doing great, you?"),
    ("Hi, what's good?", "Hey! All good here!"),
    ("Is it cold out there?", "A little chilly, bundle up!"),
    ("What do you enjoy doing?", "I like learning, what's your hobby?"),
    ("See you later!", "Bye!"),
    ("How are you today?", "I'm fantastic, thanks!"),
    ("Hey, how's it going?", "Yo! I'm good, you?"),
    ("What's the weather like today?", "It's warm and sunny!"),
    ("What do you do for fun?", "I love chatting, you?"),
    ("Take care!", "You too!"),
    ("How's your mood today?", "I'm in a great mood, you?"),
    ("Good evening!", "Evening! Hope you're having a nice night."),
    ("Is it nice outside?", "Yeah, it's beautiful out!"),
    ("What's your favorite hobby?", "I enjoy exploring, you?"),
    ("Bye bye!", "See ya!"),
    ("How's your day going?", "Pretty awesome, you?"),
    ("Hi there!", "Hey! What's up?"),
    ("What's the weather doing?", "It's clear and cool!"),
    ("What do you like to do?", "I love a good convo, you?"),
    ("Have a great one!", "You too!"),
    ("How you feeling today?", "I'm great, thanks!"),
    ("Hello, friend!", "Hey! Good to hear from you."),
    ("Is it sunny today?", "Yup, sun's out!"),
    ("What are your hobbies?", "I like learning, you?"),
    ("Catch you later!", "Until next time!"),
    ("How's it hanging?", "All good, you?"),
    ("Good afternoon!", "Afternoon! Hope your day's going well."),
    ("What's the weather like now?", "It's nice and breezy!"),
    ("Got any cool interests?", "I love chatting, what's yours?"),
    ("Talk later!", "Bye for now!"),
    ("How are you doing?", "I'm awesome, thanks!"),
    ("Hey, what's good?", "Yo! All good here!"),
    ("Is it warm out?", "Yeah, it's nice and toasty!"),
    ("What do you enjoy?", "I like helping out, you?"),
    ("See ya later!", "Later!"),
    ("How's your vibe?", "Super chill, you?"),
    ("Hi, what's new?", "Not much, just here for you!"),
    ("What's the weather today?", "It's sunny and warm!"),
    ("What do you like doing?", "I enjoy a good chat, you?"),
    ("Have a good day!", "You too!"),
    ("How you doing today?", "I'm fantastic, thanks!"),
    ("Hello!", "Hey! What's up?"),
    ("Is it nice out?", "Yeah, perfect day out there!"),
    ("What's your favorite thing to do?", "I love learning, you?"),
    ("Bye for now!", "Catch you later!"),
    ("How's your day been?", "It's been great, you?"),
    ("Hey there!", "Yo! What's good?"),
    ("What's the weather like?", "It's cool and sunny!"),
    ("Got any fun hobbies?", "I like exploring, you?"),
    ("Take it easy!", "You too!"),
    ("How you feeling?", "I'm great, thanks!"),
    ("Good morning!", "Morning! Hope you're ready for the day!"),
    ("Is it rainy out?", "Nope, clear skies!"),
    ("What do you do for fun?", "I love chatting, you?"),
    ("See you soon!", "Later!"),
    ("How's everything going?", "All good, you?"),
    ("Hi, how you doing?", "Hey! I'm awesome!"),
    ("What's the weather today?", "It's warm and pleasant!"),
    ("What are your interests?", "I like learning, you?"),
    ("Goodbye!", "See ya!"),
    ("How's your day?", "Going great, thanks!"),
    ("Hey, what's up?", "Not much, just here!"),
    ("Is it hot out there?", "A bit toasty, but nice!"),
    ("What's your favorite activity?", "I enjoy helping out, you?"),
    ("Have a good night!", "Sweet dreams!"),
    ("How you holding up?", "I'm good, you?"),
    ("Hello there!", "Hey! Nice to see you!"),
    ("What's the weather like?", "It's sunny and nice!"),
    ("Got any hobbies?", "I love a good chat, you?"),
    ("Talk soon!", "Later!"),
    ("How's it going today?", "I'm doing great, you?"),
    ("Hi, what's good?", "Hey! All good here!"),
    ("Is it cold today?", "A little chilly, grab a coat!"),
    ("What do you enjoy doing?", "I like learning, you?"),
    ("See you later!", "Bye!"),
    ("How are you today?", "I'm fantastic, thanks!"),
    ("Hey, how's it going?", "Yo! I'm good, you?"),
    ("What's the weather like today?", "It's warm and sunny!"),
    ("What do you do for fun?", "I love chatting, you?"),
    ("Take care!", "You too!"),
    ("How's your mood today?", "I'm in a great mood, you?"),
    ("Good evening!", "Evening! Hope you're having a nice night."),
    ("Is it nice outside?", "Yeah, it's beautiful out!"),
    ("What's your favorite hobby?", "I enjoy exploring, you?"),
    ("Bye bye!", "See ya!"),
    ("How's your day going?", "Pretty awesome, you?"),
    ("Hi there!", "Hey! What's up?"),
    ("What's the weather doing?", "It's clear and cool!"),
    ("What do you like to do?", "I love a good convo, you?"),
    ("Have a great one!", "You too!"),
    ("How you feeling today?", "I'm great, thanks!"),
    ("Hello, friend!", "Hey! Good to hear from you.")
]

In [53]:
pairs = [(inp, "<start> " + tgt + " <end>") for inp, tgt in pairs]

In [54]:
inputs, targets = zip(*pairs)

In [55]:
inputs = [i.replace("\n", " ") for i in inputs]
targets = [t.replace("\n", " ") for t in targets]

In [56]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(inputs + targets)

In [57]:
print(tokenizer.word_index) 

{'start': 1, 'end': 2, 'you': 3, "what's": 4, 'good': 5, 'i': 6, 'your': 7, 'a': 8, 'it': 9, 'like': 10, 'do': 11, "i'm": 12, "how's": 13, 'hey': 14, 'today': 15, "it's": 16, 'how': 17, 'out': 18, 'great': 19, 'the': 20, 'what': 21, 'for': 22, 'later': 23, 'and': 24, 'weather': 25, 'doing': 26, 'thanks': 27, 'there': 28, 'nice': 29, 'day': 30, 'is': 31, 'bye': 32, 'love': 33, 'going': 34, 'up': 35, 'to': 36, 'see': 37, 'hi': 38, 'enjoy': 39, 'sunny': 40, 'are': 41, 'here': 42, 'warm': 43, 'learning': 44, 'too': 45, 'new': 46, 'hobbies': 47, 'all': 48, 'fun': 49, 'hello': 50, 'have': 51, 'any': 52, 'just': 53, 'chatting': 54, 'now': 55, 'favorite': 56, 'outside': 57, 'feeling': 58, 'ya': 59, 'cool': 60, 'hope': 61, 'awesome': 62, 'exploring': 63, 'yo': 64, 'yeah': 65, 'morning': 66, 'been': 67, 'take': 68, 'catch': 69, 'got': 70, 'not': 71, 'much': 72, 'hobby': 73, 'night': 74, 'soon': 75, 'mood': 76, 'friend': 77, 'asking': 78, 'clear': 79, 'chat': 80, 'helping': 81, "you're": 82, 'car

In [58]:
input_seqs = tokenizer.texts_to_sequences(inputs)
target_seqs = tokenizer.texts_to_sequences(targets)

In [59]:
max_len = 10


input_seqs = pad_sequences(input_seqs, maxlen=max_len, padding="post", truncating="post")
target_seqs = pad_sequences(target_seqs, maxlen=max_len, padding="post", truncating="post")

In [60]:
vocab_size = len(tokenizer.word_index) + 1
print(vocab_size)

173


In [61]:
input_seqs[0]

array([38,  0,  0,  0,  0,  0,  0,  0,  0,  0], dtype=int32)

In [62]:
embed_dim = 64
latent_dim = 64

In [63]:
encoder_inputs = Input(shape=(max_len,))


x = Embedding(vocab_size, embed_dim, mask_zero=True)(encoder_inputs)
encoder_outputs, state_h, state_c = LSTM(latent_dim, return_state=True)(x)

encoder_states = [state_h, state_c]

In [64]:
decoder_inputs = Input(shape=(max_len,))


y = Embedding(vocab_size, embed_dim, mask_zero=True)(decoder_inputs)


decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(y, initial_state=encoder_states)

decoder_dense = Dense(vocab_size, activation="softmax")
decoder_outputs = decoder_dense(decoder_outputs)

In [65]:
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

In [66]:
model.compile(optimizer="adam", loss="sparse_categorical_crossentropy")

In [67]:
decoder_target_data = np.expand_dims(target_seqs, -1)

In [68]:
print(decoder_target_data.shape)
print(target_seqs.shape)

(186, 10, 1)
(186, 10)


In [69]:
model.fit([input_seqs, target_seqs], decoder_target_data, batch_size=16, epochs=20, validation_split=0.1)


Epoch 1/20
11/11 ━━━━━━━━━━━━━━━━━━━━ 4s 73ms/step - loss: 5.1311 - val_loss: 5.0799
Epoch 2/20
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step - loss: 5.0191 - val_loss: 4.8016
Epoch 3/20
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - loss: 4.5225 - val_loss: 3.8222
Epoch 4/20
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - loss: 3.8266 - val_loss: 3.3422
Epoch 5/20
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - loss: 3.5480 - val_loss: 3.1251
Epoch 6/20
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - loss: 3.3395 - val_loss: 2.9950
Epoch 7/20
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step - loss: 3.2040 - val_loss: 2.8688
Epoch 8/20
11/11 ━━━━━━━━━━━━━━━━━━━━ 1s 51ms/step - loss: 3.0845 - val_loss: 2.7586
Epoch 9/20
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step - loss: 2.9812 - val_loss: 2.6594
Epoch 10/20
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step - loss: 2.8954 - val_loss: 2.5696
Epoch 11/20
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step - loss: 2.8119 - val_loss: 2.4836
Epoch 12/20
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step - loss: 2.7

In [73]:
def generate_reply(input_text, max_len=max_len):

    seq = tokenizer.texts_to_sequences([input_text.replace("\n"," ")])
    seq = pad_sequences(seq, maxlen=max_len, padding="post", truncating="post")
    
    start_token_idx = tokenizer.word_index["start"]
    end_token_idx = tokenizer.word_index["end"]

    target_seq = [start_token_idx]
    
    reverse_word_index = {idx: word for word, idx in tokenizer.word_index.items()}
    
    reply_words = []
    
    for i in range(max_len):

        decoder_input = pad_sequences([target_seq], maxlen=max_len, padding="post")
    
        output_tokens = model.predict([seq, decoder_input], verbose=0)
        
        next_idx = np.argmax(output_tokens[0, len(target_seq)-1, :])
    
        if next_idx == end_token_idx:
            break
        
        word = reverse_word_index.get(next_idx, "")
        reply_words.append(word)
        
        target_seq.append(next_idx)
    
    return " ".join(reply_words)

In [74]:
input_text = "Hi"

print("Input:", input_text)

print("Reply:", generate_reply(input_text))

Input: Hi
Reply: start start start start start start start start start start


# Real Dataset

In [2]:
df = pd.read_csv("train.csv")  

In [26]:
print(df.head())
print(df.describe())
print(df.info())
print(df.shape)

                                              dialog                    act  \
0  ['Say , Jim , how about going for a few beers ...  [3 4 2 2 2 3 4 1 3 4]   
1  ['Can you do push-ups ? '\n " Of course I can ...          [2 1 2 2 1 1]   
2  ['Can you study with the radio on ? '\n ' No ,...            [2 1 2 1 1]   
3  ['Are you all right ? '\n ' I will be all righ...              [2 1 1 1]   
4  ['Hey John , nice skates . Are they new ? '\n ...    [2 1 2 1 1 2 1 3 4]   

                 emotion  
0  [0 0 0 0 0 0 4 4 4 4]  
1          [0 0 6 0 0 0]  
2            [0 0 0 0 0]  
3              [0 0 0 0]  
4    [0 0 0 0 0 6 0 6 0]  
                                                   dialog        act  \
count                                               11118      11118   
unique                                              10549       5005   
top     ['Can I help you find something ? ' ' I would ...  [2 1 2 1]   
freq                                                    3        344   

  

In [27]:
print(df.iloc[0])
dialog=df.iloc[0][0]
print(dialog)
print(type(df['dialog'].iloc[0]))

dialog     ['Say , Jim , how about going for a few beers ...
act                                    [3 4 2 2 2 3 4 1 3 4]
emotion                                [0 0 0 0 0 0 4 4 4 4]
Name: 0, dtype: object
['Say , Jim , how about going for a few beers after dinner ? '
 ' You know that is tempting but is really not good for our fitness . '
 ' What do you mean ? It will help us to relax . '
 " Do you really think so ? I don't . It will just make us fat and act silly . Remember last time ? "
 " I guess you are right.But what shall we do ? I don't feel like sitting at home . "
 ' I suggest a walk over to the gym where we can play singsong and meet some of our friends . '
 " That's a good idea . I hear Mary and Sally often go there to play pingpong.Perhaps we can make a foursome with them . "
 ' Sounds great to me ! If they are willing , we could ask them to go dancing with us.That is excellent exercise and fun , too . '
 " Good.Let ' s go now . " ' All right . ']
<class 'str'>


C:\Users\Sina's Pc\AppData\Local\Temp\ipykernel_21832\4159947092.py:2: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  dialog=df.iloc[0][0]


In [75]:
pairs = []

for idx, row in df.iterrows():
    text = row['dialog']
    
    text = text.strip("[]")              
    text = text.replace("'", "")         
    text = text.replace('"', "")        

    utterances = [u.strip() for u in text.split(",") if u.strip() != ""]
    
    for i in range(len(utterances)-1):
        pairs.append((utterances[i], utterances[i+1]))

print(len(pairs))
print(pairs[:1])

NameError: name 'df' is not defined